In [ ]:
'''
Miguel Singson BS-CSST

References Used: 
https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/#pandas-filtering-based-on-comparsion-operator
https://www.geeksforgeeks.org/python-pandas-apply/
https://www.statology.org/pandas-mean-by-group/
'''


# Zomato Exercise

Zomato is a restaurant aggregator website. The platform provides information, menus and user-reviews of restaurants as well as food delivery options from partner restaurants in select cities. You have been provided 3 datasets:

- zomato.csv contains restaurant information like average_cost_for_two, cuisines
- location.csv contains location of the restaurants like address, city, locality & etc
- ratings.csv contains aggregate rating for each restaurant

The different files are connected using the `id` columns which can be found in the different files. The id column is a unique identifier to represent a restaurant. You can use this to **merge** the different files.

In [20]:
import pandas as pd
import numpy as np

1. Merge the zomato, location & ratings file into a signle dataframe and store it to the `merged_df` variable **(3pts)**

In [21]:
merged_df = pd.merge(pd.read_csv('zomato.csv'), pd.read_csv('location.csv'), on='id')
merged_df = pd.merge(merged_df, pd.read_csv('ratings.csv'), on='id')

In [22]:
assert (6830, 35)==merged_df.shape

cols = ['id', 'address', 'city', 'city_id', 'country_id', 'latitude',
       'locality', 'locality_verbose', 'longitude', 'zipcode',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes',
       'average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view',
       'is_delivering_now', 'is_table_reservation_supported',
       'is_zomato_book_res', 'menu_url', 'mezzo_provider', 'name',
       'photos_url', 'price_range', 'thumb', 'url']

assert sorted(merged_df.columns) == sorted(cols)

2. Create a function that returns the average aggregate rating for each city. Expected returned value is a Series datatype which contains name of city as index and average aggregate rating as value. **(2pts)**

In [23]:
def exercise_2(merged_df):
    result = merged_df.groupby(['city'])['aggregate_rating'].mean()
    return result

In [24]:
res_series = exercise_2(merged_df)
assert True == np.isclose(3.302075, res_series[res_series.index == 'Pasay City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.890741, res_series[res_series.index == 'Valenzuela City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]
assert True == np.isclose(1.558333, res_series[res_series.index == 'Malabon City'],
                          rtol=1e-05, atol=1e-08, equal_nan=False)[0]

3. Using the apply function of pandas convert the rating_text to a numeric equivalent. You are to introduce a new column in merged_df called `rating_numeric` which is based on the value from rating_text: **(3pts)**

Excellent = 5 <br>
Very Good = 4 <br>
Good = 3 <br>
Average = 2 <br>
Poor = 1 <br>
Not rated = 0 <br>

In [25]:
def rate_num(rating):
    if rating == 'Excellent':
        return 5
    elif rating == 'Very Good':
        return 4
    elif rating == 'Good':
        return 3
    elif rating == 'Average':
        return 2
    elif rating == 'Poor':
        return 1
    elif rating == 'Not rated':
        return 0
        
merged_df['rating_numeric'] = merged_df['rating_text'].apply(rate_num)

In [26]:
assert 868 == merged_df[merged_df.rating_numeric == 0].shape[0]
assert True == ('rating_numeric' in merged_df.columns)

4. Create a function which filters the merged dataframe to only contain rows coming in from a specific city. Expected returned value is a dataframe **(2pts)**

In [27]:
def exercise_4(merged_df, city):
    filter = merged_df[merged_df['city'] == city]
    return filter

In [28]:

assert 'Makati City' == exercise_4(merged_df, 'Makati City').city.unique()[0]
assert (976, 36) == exercise_4(merged_df, 'Makati City').shape

assert 'Mandaluyong City' == exercise_4(merged_df, 'Mandaluyong City').city.unique()[0]
assert (443, 36) == exercise_4(merged_df, 'Mandaluyong City').shape

assert (0, 36) == exercise_4(merged_df, 'Quezon  City').shape

5. Create a function to filter the dataframe to only contain certain columns passed in the function. Expected returned value is a dataframe **(1pt)**

In [29]:
def exercise_5(merged_df, cols_to_filter):
    filter = merged_df[(cols_to_filter)]
    return filter

In [30]:
cols = ['name', 'cuisines', 'city', 'aggregate_rating']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 4) == exercise_5(merged_df, cols).shape

cols = ['average_cost_for_two', 'book_url', 'cuisines', 'currency', 'deeplink',
       'events_url', 'featured_image', 'has_online_delivery',
       'has_table_booking', 'include_bogo_offers', 'is_book_form_web_view']
assert sorted(cols) == sorted(exercise_5(merged_df, cols).columns)
assert (6830, 11) == exercise_5(merged_df, cols).shape


6.  What is the most common restaurant name in the dataset? **(2pts)** Write down your answer and show code used to come up with the answer.

In [31]:
result = merged_df.value_counts('name')
result

#Answer: Starbucks

name
Starbucks                         99
Jollibee                          77
McDonald's                        66
KFC                               54
BonChon Chicken                   49
                                  ..
HotBox Kitchen                     1
Hotel Bar                          1
Hotties Gourmet                    1
House Manila - Remington Hotel     1
¡VENGO!                            1
Name: count, Length: 3286, dtype: int64

7. Which restaurant has the most expensive average cost for two? **(2pts)** Write down your answer and show code used to come up with the answer.

In [32]:
result = merged_df.groupby(['name'])['average_cost_for_two'].mean()
result = result.sort_values(ascending=False)
result

#Answer: The Tasting Room

name
The Tasting Room                             10000.0
Spiral - Sofitel Philippine Plaza Manila      6000.0
Medley Buffet - Okada Manila                  5400.0
Wolfgang's Steakhouse by Wolfgang Zwiener     5100.0
Yakumi - Solaire Resort & Casino              5000.0
                                              ...   
Sari Salo                                        0.0
La Vita at Marina Bay                            0.0
Bun Appetit                                      0.0
Scarlet Wine Lounge                              0.0
Propaganda Vietnamese Bistro                     0.0
Name: average_cost_for_two, Length: 3286, dtype: float64

8. How many restaurants do not have a zipcode? **(1pt)** Write down your answer and show code used to come up with the answer.

In [33]:
merged_df['zipcode'].isnull().sum()

#Answer: 6713

6713

9. How many restaurants in the dataset allow online delivery? **(1pt)** Write down your answer and show code used to come up with the answer.

In [34]:
merged_df['has_online_delivery'].sum()

#Answer: 0

0

10. Which locatlity has more expensive food? Poblacion or Salcedo Village? **(3pts)** Write down your answer and show code used to come up with the answer.


In [35]:
options = ['Poblacion', 'Salcedo Village']
cols_to_filter = ['locality', 'average_cost_for_two']
result = merged_df[(merged_df['locality'].isin(options))]
result = result.groupby(['locality'])['average_cost_for_two'].mean()

result

#Answer: Salcedo Village

locality
Poblacion          898.214286
Salcedo Village    989.102564
Name: average_cost_for_two, dtype: float64